In [ ]:
from update_leistungsdaten import get_day_df_long, DownloadError, ParseError
import requests
import re
from io import StringIO
from datetime import datetime
from typing import List, Tuple
from tqdm import tqdm
import datetime as dt
import polars as pl


def update_leistung(end = dt.datetime.now()):
    standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof",
                 "karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]
    
    days_back = 5
    start = end - dt.timedelta(days=days_back)
    required_dates = [start + dt.timedelta(days=i) for i in range((end - start).days + 1)]

    for s in tqdm(standorte):
        missing_dates = required_dates
        for date in missing_dates:
            print(s,date)
            try:
                temp = get_day_df_long(s, date.strftime("%y%m%d"))
                temp["standort"] = s
            except DownloadError as de:
                print(de)
                
            except ParseError as pe:
                print(pe)
        
            
            
                
        


In [ ]:
from datetime import datetime, date as date_type
from update_leistungsdaten import get_day_df_long, DownloadError, ParseError
import datetime as dt
from zoneinfo import ZoneInfo
import polars as pl
import os


def get_data(standort, date):
    path = "./test_delta/"
            
    today = datetime.now(tz=ZoneInfo("Europe/Berlin")).date()
    if date==today:
        print("date=heute: rufe aktuelle Daten ab!")
        return get_day_df_long(standort, date.strftime("%y%m%d"))
    if not os.path.exists(path):
        print("noch kein delta vorhanden --> fetch data und neues delta anlegen")
        df = get_day_df_long(standort, date.strftime("%y%m%d"))
        pl.DataFrame(df).write_delta(path)
        
        return df
    else:
        print("date!=heute: prüfe ob daten im deltalake vorhanden?")
        lazy = pl.scan_delta(path)
        filtered_lazy =lazy.filter(pl.col("Datetime").dt.date() == date)
        return lazy
            
        
        
temp = get_data("badboll",dt.datetime.now().date()- dt.timedelta(days=1))



In [ ]:
df = get_day_df_long("badboll", datetime.now(tz=ZoneInfo("Europe/Berlin")).date().strftime("%y%m%d"))

In [ ]:
pl.DataFrame(df).write_delta("/Users/simon/Desktop/OEEG Dashboard/test_delta")

In [ ]:
pl.read_delta("/Users/simon/Desktop/OEEG Dashboard/test_delta")#.collect()

In [1]:
from backend_leistung import optimize, get_day_and_update,get_heutige_Leistung
import datetime as dt

standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof",
                "karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]

end = dt.datetime.now()
days_back = 50
start = end - dt.timedelta(days=days_back)
required_dates = [start + dt.timedelta(days=i) for i in range((end - start).days + 1)]

new_data = []
for s in standorte:
    missing_dates = required_dates
    for date in missing_dates:
       _ = get_day_and_update(s, date.date()) 
       
optimize()

[getDay]: badboll - 2025-11-29 - heute... Downloaded shape: (1392, 6)
[getDay]: esslingen - 2025-11-29 - heute... Downloaded shape: (1488, 6)
[getDay]: geislingen - 2025-11-29 - heute... Downloaded shape: (112, 6)
[getDay]: holzgerlingen - 2025-11-29 - heute... Downloaded shape: (384, 6)
[getDay]: hospitalhof - 2025-11-29 - heute... Downloaded shape: (1232, 6)
[getDay]: karlsruhe - 2025-11-29 - heute... Downloaded shape: (612, 6)
[getDay]: mettingen - 2025-11-29 - heute... Downloaded shape: (448, 6)
[getDay]: muensingen - 2025-11-29 - heute... Downloaded shape: (5250, 6)
[getDay]: tuebingen - 2025-11-29 - heute... Downloaded shape: (2576, 6)
heute: ParseError: regex https://www.oekumenische-energiegenossenschaft.de/datenlogger/waiblingen/visualisierung/min_day.js
[getDay]: waiblingen - 2025-11-29 - heute... Downloaded shape: (0, 0)
[optimize]: Files before: 1, after: 1


In [2]:
from backend_leistung import get_heutige_Leistung

get_heutige_Leistung("geislingen") 

[getDay]: geislingen - 2025-11-29 - heute... Downloaded shape: (112, 6)


array([], dtype=int64)

In [3]:
from update_ertragsdaten import update_ertrag
# from update_leistungsdaten import update_leistung  
update_ertrag()

Ertragsdaten aktualisiert!


In [9]:
pd.Timestamp.today() - pd.DateOffset(years=1)

Timestamp('2024-11-30 11:25:23.167983')

In [19]:
import pandas as pd

ertrag = (
    pd.read_parquet("app/data/ertrag.parquet")
      .assign(date=lambda df: pd.to_datetime(df["date"]).dt.date)
      .loc[lambda df: df["standort"] == "muensingen"]
)

start = (pd.Timestamp.today() - pd.DateOffset(years=1)).date()
end   = pd.Timestamp.today().date()

ertrag = (
    ertrag.loc[lambda df: df["date"] >= start]
          .groupby("date").sum()
          .reindex(pd.date_range(start, end, freq="D"))
          .rename_axis("date")
          .reset_index()
          .assign(value=lambda df: df["value"].fillna(0))   # <-- Fill
)
ertrag

,date,standort,wr,value
0,2024-11-30,NaN,NaN,0.0
1,2024-12-01,NaN,NaN,0.0
2,2024-12-02,NaN,NaN,0.0
3,2024-12-03,NaN,NaN,0.0
4,2024-12-04,NaN,NaN,0.0
...,...,...,...,...
361,2025-11-26,muensingenmuensingenmuensingenmuensingenmuensi...,21.0,10149.0
362,2025-11-27,muensingenmuensingenmuensingenmuensingenmuensi...,21.0,13573.0
363,2025-11-28,muensingenmuensingenmuensingenmuensingenmuensi...,21.0,26576.0
364,2025-11-29,muensingenmuensingenmuensingenmuensingenmuensi...,21.0,43938.0


In [10]:
ertrag.sort_values("date")

,date,standort,wr,value
81878,2025-01-23,muensingen,2,11584
81887,2025-01-23,muensingen,5,8109
81888,2025-01-23,muensingen,4,3753
81889,2025-01-23,muensingen,3,13673
81890,2025-01-23,muensingen,1,12340
...,...,...,...,...
86845,2025-11-30,muensingen,2,1519
86846,2025-11-30,muensingen,1,1404
86848,2025-11-30,muensingen,4,1248
86849,2025-11-30,muensingen,5,1405
